## Loading of Miller ECoG data of faces/houses (+ noise)

includes some visualizations

In [22]:
#@title Data retrieval
import os, requests

fname = 'faceshouses.npz'
url = "https://osf.io/78saz/download"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("!!! Failed to download data !!!")
  else:
    if r.status_code != requests.codes.ok:
      print("!!! Failed to download data !!!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)


In [23]:
#@title Import matplotlib and set defaults
from matplotlib import rcParams 
from matplotlib import pyplot as plt
rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] =15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

In [65]:
#@title Data loading
import numpy as np

alldat = np.load(fname, allow_pickle=True)
alldat1 = alldat['dat1']
alldat2 = alldat['dat2']

# select just one of the recordings here. 11 is nice because it has some neurons in vis ctx. 
dat1 = alldat1[1]
dat2 = alldat2[1]

print(dat1.keys())
print(dat2.keys())

dict_keys(['t_off', 'stim_id', 't_on', 'srate', 'V'])
dict_keys(['stim_id', 'stim_cat', 'stim_noise', 't_on', 't_off', 'key_press', 'V', 'categories'])


# Dataset info #

This is one of multiple ECoG datasets from Miller 2019, recorded in a clinical settings with a variety of tasks. We plan to curate a few more before NMA starts. Raw data here:

https://exhibits.stanford.edu/data/catalog/zk881ps0522

`dat1` and `dat2` contain 5 sessions from 5 subjects, and was originally used in this publication: 

*Miller, Kai J., Dora Hermes, Franco Pestilli, Gagan S. Wig, and Jeffrey G. Ojemann. "Face percept formation in human ventral temporal cortex." Journal of neurophysiology 118, no. 5 (2017): 2614-2627.*

In this task, subjects in a clinical settings (with ECoG implants) are passively shown faces and house during the first experiment (`dat1`). Then in the second experiment in the same subjects (`dat2`), noise is added to face and houses images and the subject has to detect the faces by pressing a key. 

Sample rate is always 1000Hz, and the ECoG data has been notch-filtered at 60, 120, 180, 240 and 250Hz, followed by z-scoring across time and conversion to float16 to minimize size. 


Experiment 1: 
* `dat1['V']`: continuous voltage data (time by channels)
* `dat1['srate']`: acquisition rate (1000 Hz). All stimulus times are in units of this.  
* `dat1['t_on']`: time of stimulus onset in data samples
* `dat1['t_off']`: time of stimulus offset, always 400 samples after `t_on`
* `dat1['stim_id`]: identity of stimulus from 1-100, with 1-50 being houses and 51-100 being faces

Experiment 2: 
* `dat2['V`]: continuous voltage data (time by channels)
* `dat2['srate']`: acquisition rate (1000 Hz). All stimulus times are in units of this.  
* `dat2['t_on']`: time of stimulus onset in data samples
* `dat2['t_off']`: time of stimulus offset, always 1000 samples after `t_on`, with no inter-stimulus interval
* `dat2['stim_id`]: identity of stimulus from 1-600 (not really useful, since we don't know which ones are the same house/face)
* `dat2['stim_cat']`: stimulus category (1 = house, 2 = face)
* `dat2['stim_noise']`: percent noise from 0 to 100
* `dat2['key_press']`: when the subject thought the image was a face
* `dat2['categories']`: categories legend (1 = house, 2 = face)


In [66]:
# quick way to get broadband power in time-varying windows
from scipy import signal

# filter in 10-100Hz
b, a = signal.butter(3, [10, 100], btype = 'band', fs=1000)
V = dat1['V']
V = signal.filtfilt(b,a,V,0)

# absolute value and then time-varying envelope of signal, low-passed at 50Hz
V = np.abs(V)
b, a = signal.butter(3, [25], btype = 'low', fs=1000)
V = signal.filtfilt(b,a,V,0)

In [67]:
# average the broadband power across all face stimuli and across all house stimuli

nt, nchan = V.shape
nstim = len(dat1['t_on'])

trange = np.arange(-400, 400)
ts = dat1['t_on'][:,np.newaxis] + trange
V_epochs = np.reshape(V[ts, :], (nstim, 800, nchan))

V_house = (V_epochs[dat1['stim_id']<=50]).mean(0)
V_face  = (V_epochs[dat1['stim_id']>50]).mean(0)

In [ ]:
# let's find the electrodes that distinguish faces from houses
from matplotlib import pyplot as plt

plt.figure(figsize=(20,10))
for j in range(50):
  ax = plt.subplot(5,10,j+1)
  plt.plot(trange, V_house[:,j])
  plt.plot(trange, V_face[:,j])
  plt.title('ch%d'%j)
  plt.xticks([-400, 0, 400])
  plt.ylim([0, .7])

In [ ]:
# let's look at all the face trials for electrode 46 that has a good response to faces
plt.subplot(1,3,1)
plt.imshow(V_epochs[dat1['stim_id']>50,:,46].astype('float32'), aspect='auto')
plt.colorbar()

In [ ]:
# Electrode 44 is also interesting: it ramps up in anticipation of the stimulus 
# and seems to decay back after the stimulus is identified
# This happens for both face and house trials: 

plt.subplot(1,3,1)
plt.imshow(V_epochs[:,:,44].astype('float32'), aspect='auto')
plt.colorbar()

In [71]:
# this is still work in progress! need to do a few more visualizations and we'll probably need to change the way the broadband power is computed to get better signals. 